## Preprocessing

In [4]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Drop the non-beneficial ID columns ('EIN', 'NAME') and the column with no variability ('STATUS')
application_df = application_df.drop(columns=['EIN', 'NAME', 'STATUS'])

# Display the first few rows of the updated DataFrame
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,142590,1


In [6]:
# Group rare APPLICATION_TYPE values into 'OTHER'
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
rare_types = application_type_counts[application_type_counts < 500].index  # adjust the threshold if needed
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(rare_types, 'OTHER')

In [7]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(app_type_counts)

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
OTHER      276
Name: count, dtype: int64


In [8]:
# Group rare CLASSIFICATION values into 'OTHER'
classification_counts = application_df['CLASSIFICATION'].value_counts()
rare_classifications = classification_counts[classification_counts < 1000].index  # adjust threshold if needed
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(rare_classifications, 'OTHER')


In [9]:
# Apply log transformation to ASK_AMT to compress its range
application_df['ASK_AMT_LOG'] = np.log1p(application_df['ASK_AMT'])  # log(1 + x) to avoid log(0) issues

In [10]:
# Define bin edges for ASK_AMT
bins = [0, 5000, 25000, 100000, 1000000, application_df['ASK_AMT'].max()]
labels = ['Low', 'Moderate', 'High', 'Very High', 'Extreme']

# Bin the ASK_AMT values
application_df['ASK_AMT_BINNED'] = pd.cut(application_df['ASK_AMT'], bins=bins, labels=labels)

In [11]:
# Check the updated dataset and number of unique values
print(application_df.nunique())
application_df.head()

APPLICATION_TYPE             9
AFFILIATION                  6
CLASSIFICATION               6
USE_CASE                     5
ORGANIZATION                 4
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
ASK_AMT_LOG               8747
ASK_AMT_BINNED               5
dtype: int64


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,ASK_AMT_LOG,ASK_AMT_BINNED
0,T10,Independent,C1000,ProductDev,Association,0,N,5000,1,8.517393,Low
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,108590,1,11.595344,Very High
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0,8.517393,Low
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,6692,1,8.808817,Moderate
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,142590,1,11.867736,Very High


In [12]:
# Separate features (X) and target (y)
X = application_df.drop(columns=['IS_SUCCESSFUL', 'ASK_AMT', 'ASK_AMT_BINNED'])  # Exclude target and any non-useful columns
y = application_df['IS_SUCCESSFUL']

In [13]:
# One-hot encode categorical columns
X = pd.get_dummies(X)

In [14]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
scaler = StandardScaler()

# Fit and transform the training data, then transform the test data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.


nn = tf.keras.models.Sequential()

# First hidden layer
# Let's assume we have 10 features after scaling and decide to start with 128 nodes
nn.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim=X_train_scaled.shape[1]))

# Second hidden layer
# You can experiment with 64 nodes
nn.add(tf.keras.layers.Dense(units=64, activation='relu'))

# Output layer
# For binary classification, use 1 output unit with sigmoid activation
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │           5,504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,825 (54.00 KB)

 Trainable params: 13,825 (54.00 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Compile the model
nn.compile(optimizer='adam',               # Adam optimizer is widely used
           loss='binary_crossentropy',      # Binary crossentropy for binary classification
           metrics=['accuracy'])           # Accuracy as the evaluation metric

In [19]:
# Train the model
history = nn.fit(X_train_scaled, y_train,
                 epochs=50,        # Number of training iterations (adjust as needed)
                 batch_size=32,     # Number of samples per gradient update (adjust as needed)
                 validation_data=(X_test_scaled, y_test),  # Use validation data for evaluation
                 verbose=2)         # Set verbose to 2 to print the progress bar and loss/accuracy at each epoch

Epoch 1/50
858/858 - 4s - 4ms/step - accuracy: 0.7235 - loss: 0.5647 - val_accuracy: 0.7227 - val_loss: 0.5627
Epoch 2/50
858/858 - 1s - 2ms/step - accuracy: 0.7290 - loss: 0.5525 - val_accuracy: 0.7220 - val_loss: 0.5587
Epoch 3/50
858/858 - 3s - 3ms/step - accuracy: 0.7318 - loss: 0.5497 - val_accuracy: 0.7249 - val_loss: 0.5570
Epoch 4/50
858/858 - 3s - 3ms/step - accuracy: 0.7336 - loss: 0.5471 - val_accuracy: 0.7270 - val_loss: 0.5575
Epoch 5/50
858/858 - 4s - 4ms/step - accuracy: 0.7332 - loss: 0.5467 - val_accuracy: 0.7238 - val_loss: 0.5554
Epoch 6/50
858/858 - 1s - 2ms/step - accuracy: 0.7331 - loss: 0.5457 - val_accuracy: 0.7246 - val_loss: 0.5584
Epoch 7/50
858/858 - 2s - 3ms/step - accuracy: 0.7328 - loss: 0.5443 - val_accuracy: 0.7267 - val_loss: 0.5581
Epoch 8/50
858/858 - 1s - 2ms/step - accuracy: 0.7339 - loss: 0.5441 - val_accuracy: 0.7261 - val_loss: 0.5551
Epoch 9/50
858/858 - 1s - 2ms/step - accuracy: 0.7356 - loss: 0.5433 - val_accuracy: 0.7255 - val_loss: 0.5578
E

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 2ms/step - accuracy: 0.7246 - loss: 0.5686
Loss: 0.5686216950416565, Accuracy: 0.7246355414390564


In [21]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')